<img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px;" />

# 2.4 Formats and Performance

When executing code on your local machine or in the cloud, it is important to remember that each operation uses resources.

In this section we showcase a python library `codecarbon` that can be used to estimate the energy consumption of your code.

Please note that benchmarking systems and profiling code is a complex topic with many variables. These examples merely illustrate very rough estimates.

Start by import all the necessary libraries and utilities.

In [2]:
import random
import numpy as np
import openeo
from openeo.local import LocalConnection

from codecarbon_utils import calculate_emission_equivalents, CustomEmissionsTracker
local_conn = LocalConnection('')

Did not load machine learning processes due to missing dependencies: Install them like this: `pip install openeo-processes-dask[implementations, ml]`


Setting up the emission tracker for "offline" use to not interact with the CodeCarbon API.

Changing the country code will alter the carbon intensity value used to calculate the emissions.

To learn more about the methodology behind `codecarbon` see the [documentation](https://mlco2.github.io/codecarbon/methodology.html#carbon-intensity).

Feel free to change the country code to wherever you are to see how it affects the carbon emissions. ISO codes can be found on [Wikipedia](https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes).

In [3]:
tracker = CustomEmissionsTracker(
    country_iso_code="ITA",
    log_level="error",
    save_to_file=True,
    output_dir="./",
)

[codecarbon INFO @ 11:20:05] offline tracker init


## Simple example

This example performs a simple matrix multiplication using numpy and estimates the energy consumption of the operation.

As a comparison, we also perform a matrix multiplication using the standard library functionality of Python.

What you will see is that utilizing high performance libraries like numpy can dramatically increase the efficiency. 

In [4]:
tracker.start_experiment(experiment_id=1)

def matrix_numpy(size: int, iterations: int) -> np.ndarray:
    """
    Compute the product of a square matrix with itself for a number of iterations using numpy.

    Args:
        size (int): Size of the matrix (size x size).
        iterations (int): Number of iterations for matrix multiplication.

    Returns:
        The result of the matrix product.
    """
    matrix = np.random.rand(size, size)
    for _ in range(iterations):
        matrix = np.dot(matrix, matrix)
    return matrix

matrix_numpy(100, 100)

tracker.stop_experiment()

Starting experiment: 1
Stopped experiment: 1


In [6]:
calculate_emission_equivalents(experiment_id=1)

In [7]:
tracker.start_experiment(experiment_id=2)

def matrix_python(size: int, iterations: int) -> list:
    """
    Compute the product of a square matrix with itself for a number of iterations using pure Python.
    
    Args:
        size (int): Size of the matrix (size x size).
        iterations (int): Number of iterations for matrix multiplication.
    
    Returns:
        The result of the matrix product.
    """

    matrix = [[random.random() for _ in range(size)] for _ in range(size)]

    for _ in range(iterations):
        result = [[0] * size for _ in range(size)]
        for i in range(size):
            for j in range(size):
                for k in range(size):
                    result[i][j] += matrix[i][k] * matrix[k][j]
        matrix = result

    return matrix

matrix_python(100, 100)

tracker.stop_experiment()

Starting experiment: 2
Stopped experiment: 2


In [8]:
calculate_emission_equivalents(experiment_id=2)

## Using a real world example

In this example, we will use our tracker to estimate the energy consumption of an NDVI workflow from previous exercise.

We need to squeeze the code into a single cell to be able to track the energy consumption more easily.

For more detailed explanation of the workflow, please refer to the previous exercise.

Overview of the workflow:
1. We select  Sentinel-2 data for a specific area and time frame of 1 year.

2. We perform temporal aggregation to get monthly composites.

3. We calculate the NDVI for each composite.

In [9]:
tracker.start_experiment(experiment_id=3)

url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a"

spatial_extent = {"west": 11.4, "east": 11.42, "south": 45.5, "north": 45.52}
temporal_extent = ["2023-01-01", "2023-12-31"]
bands = ["red","nir"]

s2_cube = local_conn.load_stac(url=url,
   spatial_extent=spatial_extent,
   temporal_extent=temporal_extent,
   bands=bands
)

s2_monthly_mean = s2_cube.aggregate_temporal_period(period="month", reducer="mean")

red = s2_monthly_mean.band("red")
nir = s2_monthly_mean.band("nir")

ndvi = (nir - red) / (nir + red)

ndvi.execute().compute()

tracker.stop_experiment()

Starting experiment: 3


/home/jzvolensky/miniconda3/envs/cubes-and-clouds/lib/python3.11/site-packages/xarray/core/groupby.py:508: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  index_grouper = pd.Grouper(


Stopped experiment: 3


In [10]:
calculate_emission_equivalents(experiment_id=3)